In [2]:
import sqlite3
import geopandas as gp

In [169]:
conn = sqlite3.connect('taxi_db_s.db')
c = conn.cursor()

In [170]:
c.execute("create table if not exists Date_time as select rowid as ID, substr(tpep_pickup_datetime,1,10) as PU_Date, substr(tpep_pickup_datetime,12,2) as PU_hour,substr(tpep_pickup_datetime,15,2) as PU_minute, substr(tpep_pickup_datetime,21,2) as PU_AM_PM, substr(tpep_dropoff_datetime,1,10) as DO_Date, substr(tpep_dropoff_datetime,12,2) as DO_hour, substr(tpep_dropoff_datetime,15,2) as DO_minute, substr(tpep_dropoff_datetime,21,2) as DO_AM_PM from taxi_details")

In [171]:
c.execute("create table if not exists taxiDetails as select rowid as ID,VendorID, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag,PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount from taxi_details")

In [174]:
c.execute("select count(ID) from Date_time")

In [175]:
a = c.fetchall()
a


[(1008970,)]

In [7]:
shp_file = gp.read_file('taxi_zones.shp')


In [20]:
from shapely.geometry import Point
count = 0           
count1 = 0       
count2 = 0
#for index, row in shp_file.iterrows():
#     for pt in list(row['geometry'].exterior.coords): 
#        print(Point(pt))
        

#error for the above loop - 'MultiPolygon' object has no attribute 'exterior' - got coordinates for 232 out of 263 locations

#checking for polygons and multipolygons

for index, row in shp_file.iterrows():
    if row['geometry'].geom_type == 'Polygon':
        count1 +=1
        row['geometry'].exterior.coords.xy[0]
        print(row['geometry'].exterior.coords[0])
    if row['geometry'].geom_type == 'MultiPolygon':
        count +=1
        for i in row['geometry']:
            count2 += 1
            print(i.exterior.coords[0])

            
print(count) 
print(count1)
print(count2)

(933100.9183527103, 192536.08569720192)
(1033269.2435912937, 172126.0078125)
(1026656.6329956204, 169555.49920654297)
(1040934.0046386868, 169654.8316040337)
(1029705.1472168118, 169249.57098388672)
(1036136.4136352539, 168588.52563475072)
(1026623.7164307088, 168509.4774169922)
(1020106.508605957, 164080.68719485402)
(1040367.3729858547, 167923.04803466797)
(1047128.450868398, 167093.0403458774)
(1047618.036966756, 166547.78679861128)
(1042096.8901977539, 166070.71618652344)
(1049019.8063964993, 163391.24340820312)
(1027824.5079956055, 164373.92138671875)
(1024442.2380371541, 163782.8989868015)
(1035865.3439941257, 163198.02600097656)
(1035624.6404826939, 161723.35316969454)
(1020440.2044067681, 160282.23919677734)
(1041392.8402099758, 160756.18939208984)
(1030776.1293945163, 160077.32940673828)
(1032542.346191436, 160994.63061524928)
(1021172.5267944634, 160840.05059815943)
(1032235.0436401516, 160805.2933959812)
(1035424.7268066257, 160278.89080809057)
(1036065.7152099758, 159517.71

In [162]:
from shapely.geometry import Polygon


x, y = shp_file.geometry[0].exterior.coords.xy

In [163]:
len(x)

232

In [ ]:
for index, row in gdf.iterrows():
     for pt in list(row['geometry'].exterior.coords): 
        print Point(pt)

In [1]:
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import random
import itertools
import math
import shapefile
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
import matplotlib as mpl
import matplotlib.pyplot as plt
import sqlite3 as db

plt.style.use('ggplot')

def get_lat_lon(sf):
    content = []
    for sr in sf.shapeRecords():
        shape = sr.shape
        rec = sr.record
        loc_id = rec[shp_dic['LocationID']]
        
        x = (shape.bbox[0]+shape.bbox[2])/2
        y = (shape.bbox[1]+shape.bbox[3])/2
        
        content.append((loc_id, x, y))
    return pd.DataFrame(content, columns=["LocationID", "longitude", "latitude"])

sf = shapefile.Reader("C:\\Users\\nikita\\Downloads\\taxi_zones/taxi_zones.shp")
fields_name = [field[0] for field in sf.fields[1:]]
shp_dic = dict(zip(fields_name, list(range(len(fields_name)))))
attributes = sf.records()
shp_attr = [dict(zip(fields_name, attr)) for attr in attributes]
df_loc = pd.DataFrame(shp_attr).join(get_lat_lon(sf).set_index("LocationID"), on="LocationID")

df_loc.to_csv("C:\\Users\\nikita\\Downloads\\taxi_zones/outLatLong.csv")
df_latlon = pd.read_csv("C:\\Users\\nikita\\Downloads\\taxi_zones/outLatLong.csv")
df_latlon.head()


,Unnamed: 0,LocationID,OBJECTID,Shape_Area,Shape_Leng,borough,zone,longitude,latitude
0,0,1,1,0.000782,0.116357,EWR,Newark Airport,9.366817e+05,190522.130278
1,1,2,2,0.004866,0.433470,Queens,Jamaica Bay,1.033536e+06,161853.982300
2,2,3,3,0.000314,0.084341,Bronx,Allerton/Pelham Gardens,1.027136e+06,254730.010849
3,3,4,4,0.000112,0.043567,Manhattan,Alphabet City,9.904240e+05,203100.040432
4,4,5,5,0.000498,0.092146,Staten Island,Arden Heights,9.321332e+05,139954.541936


In [3]:
normalised = df_latlon[['LocationID', 'longitude', 'latitude']].copy()
normalised.to_csv("C:\\Users\\nikita\\Downloads\\taxi_zones/norm_lat.csv")